# Recherche Predictive - Vision 2026 (Version Hybride)

Ce notebook presente l'approche hybride **Linear Regression + XGBoost** pour resoudre les problemes de sous-estimation des pics de decembre.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
import joblib
import os

In [ ]:
# Data preparation
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
daily_data = df_adm.groupby('date_entree').size().rename('admissions').reset_index()
daily_data = daily_data.set_index('date_entree').asfreq('D', fill_value=0)

def create_features(df):
    df = df.copy()
    df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)
    df['day_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)
    df['day_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)
    df['dayofyear'] = df.index.dayofyear
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    df['lag1'] = df['admissions'].shift(1)
    df['lag7'] = df['admissions'].shift(7)
    df['lag14'] = df['admissions'].shift(14)
    df['roll_mean_7'] = df['admissions'].shift(1).rolling(window=7).mean()
    df['roll_std_7'] = df['admissions'].shift(1).rolling(window=7).std()
    return df.dropna()

features_df = create_features(daily_data)
FEATURES = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'dayofyear', 
            'weekofyear', 'lag1', 'lag7', 'lag14', 'roll_mean_7', 'roll_std_7']
TARGET = 'admissions'

train = features_df.iloc[:-30]
test = features_df.iloc[-30:]
X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]

## Entrainement Hybride
Le modele Lineaire capture la tendance et l'echelle saisonniere, tandis qu'XGBoost corrige les erreurs (residus) locales.

In [ ]:
# 1. Linear Model (Baseline)
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_test_preds = lr.predict(X_test)

# 2. XGBoost Residual Model
residuals_train = y_train - lr.predict(X_train)
xgb_reg = xgb.XGBRegressor(n_estimators=500, learning_rate=0.03, max_depth=6)
xgb_reg.fit(X_train, residuals_train)

# 4. Final Prediction
y_pred = lr_test_preds + xgb_reg.predict(X_test)

print(f"MAE Mondiale: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index, y=y_test, name='Réel (Décembre)', line=dict(color='#005ba1')))
fig.add_trace(go.Scatter(x=test.index, y=y_pred, name='Hybride (LR+XGB)', line=dict(color='#c8102e', dash='dash')))
fig.update_layout(title='Excellence de la Prediction Hybride', template='plotly_dark')
fig.show()